# Understanding Injected Tool Arguments in LangChain

This notebook demonstrates how LangChain handles **injected tool arguments** - parameters that are automatically provided at runtime but hidden from the LLM.

## What are Injected Tool Arguments?

When building LangGraph agents, you often need tools to access:
- **Agent state** - shared data across the graph
- **Tool call IDs** - for tracking and message correlation
- **Other runtime context** - configuration, resources, etc.

But the LLM shouldn't see these parameters! The LLM should only see the "business logic" parameters.

**Solution:** Use `InjectedState`, `InjectedToolCallId`, and `InjectedToolArg` annotations.

## Setup: Import Required Libraries

In [1]:
import json
from typing import Annotated

from langchain_core.tools import InjectedToolCallId, tool
from langgraph.prebuilt import InjectedState
from typing_extensions import TypedDict

## Define a Simple Agent State

In [2]:
class AgentState(TypedDict):
    """Simple agent state containing a counter and messages."""

    counter: int
    messages: list[str]

## Example 1: Tool with ONLY Injected Arguments

This tool takes no arguments from the LLM - all parameters are injected at runtime.

In [3]:
def get_counter(
    state: Annotated[AgentState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId],
) -> str:
    """Get the current counter value from agent state.

    Returns the current counter value.

    Args:
        state: Agent state containing the counter (injected)
        tool_call_id: Tool call identifier (injected)
    """
    return f"Current counter: {state.get('counter', 0)}"

## Example 2: Tool with MIXED Arguments

This tool has both regular arguments (that LLM provides) and injected arguments.

In [4]:
def add_to_counter(
    amount: int,  # LLM WILL provide this
    state: Annotated[AgentState, InjectedState],  # Injected - LLM won't see
    tool_call_id: Annotated[
        str, InjectedToolCallId
    ],  # Injected - LLM won't see
) -> str:
    """Add a value to the counter.

    Args:
        amount: The amount to add to the counter
        state: Agent state containing the counter (injected)
        tool_call_id: Tool call identifier (injected)
    """
    current = state.get("counter", 0)
    new_value = current + amount
    return f"Added {amount} to counter. New value: {new_value}"

## Example 3: Regular Tool (No Injected Arguments)

For comparison, a normal tool where all arguments come from the LLM.

In [5]:
def multiply_numbers(
    a: int,
    b: int,
) -> int:
    """Multiply two numbers together.

    Args:
        a: First number
        b: Second number
    """
    return a * b

## Inspect Raw Function Signatures

Let's look at the function annotations before converting to LangChain tools.

In [6]:
import inspect

print("get_counter signature:")
print(inspect.signature(get_counter))
print("\nadd_to_counter signature:")
print(inspect.signature(add_to_counter))
print("\nmultiply_numbers signature:")
print(inspect.signature(multiply_numbers))

get_counter signature:
(state: typing.Annotated[__main__.AgentState, <class 'langgraph.prebuilt.tool_node.InjectedState'>], tool_call_id: typing.Annotated[str, <class 'langchain_core.tools.base.InjectedToolCallId'>]) -> str

add_to_counter signature:
(amount: int, state: typing.Annotated[__main__.AgentState, <class 'langgraph.prebuilt.tool_node.InjectedState'>], tool_call_id: typing.Annotated[str, <class 'langchain_core.tools.base.InjectedToolCallId'>]) -> str

multiply_numbers signature:
(a: int, b: int) -> int


## Convert to LangChain Tools

Use the `@tool` decorator to convert functions to StructuredTool objects.

In [7]:
# Convert to tools
tool_get_counter = tool(parse_docstring=True)(get_counter)
tool_add_to_counter = tool(parse_docstring=True)(add_to_counter)
tool_multiply = tool(parse_docstring=True)(multiply_numbers)

print("✅ Tools created successfully!")
print(f"- {tool_get_counter.name}")
print(f"- {tool_add_to_counter.name}")
print(f"- {tool_multiply.name}")

✅ Tools created successfully!
- get_counter
- add_to_counter
- multiply_numbers


## Compare: Full Schema vs LLM Schema

### Tool 1: get_counter (only injected args)

In [8]:
print("FULL RUNTIME SCHEMA (args_schema):")
full_schema = tool_get_counter.args_schema.model_json_schema()
print(f"Properties: {list(full_schema['properties'].keys())}")
print(f"Required: {full_schema.get('required', [])}")

print("\nLLM-VISIBLE SCHEMA (get_input_schema):")
llm_schema = tool_get_counter.get_input_schema().model_json_schema()
print(f"Properties: {list(llm_schema['properties'].keys())}")
print(f"Required: {llm_schema.get('required', [])}")

print(
    "\n⚠️ Notice: LLM schema shows injected args because they're in the full schema."
)
print(
    "The actual filtering happens in convert_to_anthropic_tool/convert_to_openai_tool!"
)

FULL RUNTIME SCHEMA (args_schema):
Properties: ['state', 'tool_call_id']
Required: ['state', 'tool_call_id']

LLM-VISIBLE SCHEMA (get_input_schema):
Properties: ['state', 'tool_call_id']
Required: ['state', 'tool_call_id']

⚠️ Notice: LLM schema shows injected args because they're in the full schema.
The actual filtering happens in convert_to_anthropic_tool/convert_to_openai_tool!


### Tool 2: add_to_counter (mixed args)

In [9]:
print("FULL RUNTIME SCHEMA (args_schema):")
full_schema = tool_add_to_counter.args_schema.model_json_schema()
print(f"Properties: {list(full_schema['properties'].keys())}")
print(f"Required: {full_schema.get('required', [])}")

print("\nLLM-VISIBLE SCHEMA (get_input_schema):")
llm_schema = tool_add_to_counter.get_input_schema().model_json_schema()
print(f"Properties: {list(llm_schema['properties'].keys())}")
print(f"Required: {llm_schema.get('required', [])}")

FULL RUNTIME SCHEMA (args_schema):
Properties: ['amount', 'state', 'tool_call_id']
Required: ['amount', 'state', 'tool_call_id']

LLM-VISIBLE SCHEMA (get_input_schema):
Properties: ['amount', 'state', 'tool_call_id']
Required: ['amount', 'state', 'tool_call_id']


### Tool 3: multiply_numbers (no injected args)

In [10]:
print("FULL RUNTIME SCHEMA (args_schema):")
full_schema = tool_multiply.args_schema.model_json_schema()
print(f"Properties: {list(full_schema['properties'].keys())}")
print(f"Required: {full_schema.get('required', [])}")

print("\nLLM-VISIBLE SCHEMA (get_input_schema):")
llm_schema = tool_multiply.get_input_schema().model_json_schema()
print(f"Properties: {list(llm_schema['properties'].keys())}")
print(f"Required: {llm_schema.get('required', [])}")

print("\n✅ Both are identical since there are no injected args!")

FULL RUNTIME SCHEMA (args_schema):
Properties: ['a', 'b']
Required: ['a', 'b']

LLM-VISIBLE SCHEMA (get_input_schema):
Properties: ['a', 'b']
Required: ['a', 'b']

✅ Both are identical since there are no injected args!


## Getting Clean Tool Schemas (Method 1: Anthropic)

Use `convert_to_anthropic_tool()` to get the exact schema sent to Claude.

In [11]:
from langchain_anthropic import convert_to_anthropic_tool

print("Tool 1 - get_counter (only injected args):")
schema1 = convert_to_anthropic_tool(tool_get_counter)
print(json.dumps(schema1, indent=2))

print("\n" + "=" * 60 + "\n")

print("Tool 2 - add_to_counter (mixed args):")
schema2 = convert_to_anthropic_tool(tool_add_to_counter)
print(json.dumps(schema2, indent=2))

print("\n" + "=" * 60 + "\n")

print("Tool 3 - multiply_numbers (no injected args):")
schema3 = convert_to_anthropic_tool(tool_multiply)
print(json.dumps(schema3, indent=2))

Tool 1 - get_counter (only injected args):
{
  "name": "get_counter",
  "input_schema": {
    "properties": {},
    "type": "object"
  },
  "description": "Get the current counter value from agent state. Returns the current counter value."
}


Tool 2 - add_to_counter (mixed args):
{
  "name": "add_to_counter",
  "input_schema": {
    "properties": {
      "amount": {
        "description": "The amount to add to the counter",
        "type": "integer"
      }
    },
    "required": [
      "amount"
    ],
    "type": "object"
  },
  "description": "Add a value to the counter."
}


Tool 3 - multiply_numbers (no injected args):
{
  "name": "multiply_numbers",
  "input_schema": {
    "properties": {
      "a": {
        "description": "First number",
        "type": "integer"
      },
      "b": {
        "description": "Second number",
        "type": "integer"
      }
    },
    "required": [
      "a",
      "b"
    ],
    "type": "object"
  },
  "description": "Multiply two numbers tog

### Key Observations:

1. **Tool 1** (`get_counter`): `input_schema.properties = {}` (empty!) - all args were injected
2. **Tool 2** (`add_to_counter`): Only `amount` appears - `state` and `tool_call_id` filtered out
3. **Tool 3** (`multiply_numbers`): Both `a` and `b` appear - no filtering needed

## Getting Clean Tool Schemas (Method 2: OpenAI)

Use `convert_to_openai_tool()` for a provider-agnostic approach.

In [12]:
from langchain_core.utils.function_calling import convert_to_openai_tool

print("Tool 2 - add_to_counter (OpenAI format):")
openai_schema = convert_to_openai_tool(tool_add_to_counter)
print(json.dumps(openai_schema, indent=2))

print("\n" + "=" * 60 + "\n")

# Convert to Anthropic format manually
anthropic_schema = {
    "name": openai_schema["function"]["name"],
    "description": openai_schema["function"]["description"],
    "input_schema": openai_schema["function"]["parameters"],
}

print("Same tool in Anthropic format (manual conversion):")
print(json.dumps(anthropic_schema, indent=2))

print("\n✅ Both methods produce equivalent results!")

Tool 2 - add_to_counter (OpenAI format):
{
  "type": "function",
  "function": {
    "name": "add_to_counter",
    "description": "Add a value to the counter.",
    "parameters": {
      "properties": {
        "amount": {
          "description": "The amount to add to the counter",
          "type": "integer"
        }
      },
      "required": [
        "amount"
      ],
      "type": "object"
    }
  }
}


Same tool in Anthropic format (manual conversion):
{
  "name": "add_to_counter",
  "description": "Add a value to the counter.",
  "input_schema": {
    "properties": {
      "amount": {
        "description": "The amount to add to the counter",
        "type": "integer"
      }
    },
    "required": [
      "amount"
    ],
    "type": "object"
  }
}

✅ Both methods produce equivalent results!


## How bind_tools() Works Under the Hood

Let's trace what happens when you call `model.bind_tools([tool])`.

In [13]:
# Inspect bind_tools method
import inspect

from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-sonnet-4-5-20250929", temperature=0.0)

print("bind_tools location:")
print(inspect.getfile(model.bind_tools))

# Get source code snippet
source = inspect.getsource(model.bind_tools)

# Find the conversion line
conversion_line = [
    line for line in source.split("\n") if "convert_to_anthropic_tool" in line
]
if conversion_line:
    print("\nKey line in bind_tools:")
    for line in conversion_line:
        print(line.strip())

bind_tools location:
/home/hacene/Documents/workspace/LLM/deep-agent/.venv/lib/python3.11/site-packages/langchain_anthropic/chat_models.py

Key line in bind_tools:
else convert_to_anthropic_tool(tool, strict=strict)


## The Complete Conversion Chain

```
model.bind_tools([tool])
    ↓
ChatAnthropic.bind_tools()
    ↓
convert_to_anthropic_tool(tool)
    ↓
convert_to_openai_tool(tool)  ← FILTERING HAPPENS HERE
    ↓
OpenAI format → Anthropic format
    ↓
Clean schema (no injected args)
```

In [14]:
# Verify by binding tools
bound_model = model.bind_tools([tool_add_to_counter])

print("Tools bound to model:")
print(json.dumps(bound_model.kwargs["tools"][0], indent=2))

print("\n✅ Only 'amount' appears - state and tool_call_id were filtered!")

Tools bound to model:
{
  "name": "add_to_counter",
  "input_schema": {
    "properties": {
      "amount": {
        "description": "The amount to add to the counter",
        "type": "integer"
      }
    },
    "required": [
      "amount"
    ],
    "type": "object"
  },
  "description": "Add a value to the counter."
}

✅ Only 'amount' appears - state and tool_call_id were filtered!


## Summary: Quick Reference

### Annotations for Injected Arguments

| Annotation | Purpose | From |
|------------|---------|------|
| `InjectedState` | Agent state access | `langgraph.prebuilt` |
| `InjectedToolCallId` | Tool call tracking | `langchain_core.tools` |
| `InjectedToolArg()` | Generic injection | `langchain_core.tools` |

### Getting Clean Schemas

| Method | When to Use | Returns |
|--------|-------------|----------|
| `convert_to_anthropic_tool(tool)` | Using Claude/Anthropic | Anthropic format |
| `convert_to_openai_tool(tool)` | Generic/OpenAI | OpenAI format |
| `model.bind_tools([tool])` | Runtime binding | Bound model |

### Key Insights

1. **Injected args are hidden from LLM** - they don't appear in tool schemas sent to the model
2. **Filtering happens in `convert_to_openai_tool()`** - deep in the conversion chain
3. **Runtime injection by `ToolNode`** - LangGraph's ToolNode provides injected values when executing
4. **Use `convert_to_anthropic_tool()` directly** - no need to create a model just to see the schema

### Pattern to Follow

```python
def my_tool(
    user_input: str,  # LLM provides
    state: Annotated[MyState, InjectedState],  # Auto-injected
    tool_call_id: Annotated[str, InjectedToolCallId],  # Auto-injected
) -> str:
    """Tool description.
    
    Args:
        user_input: Description for LLM
        state: Internal state (injected)
        tool_call_id: Call ID (injected)
    """
    # Use both user_input and state
    return result
```

# Part 2: Under the Hood - How ToolNode Works

Now that we understand **how to use** injected arguments, let's dive into **how they work internally**.

We'll explore:
1. How LangChain **detects** which parameters need injection
2. How **ToolNode** provides these values at runtime
3. The **complete flow** from function definition to execution

## Understanding Marker Classes

`InjectedState`, `InjectedToolCallId`, and `InjectedToolArg` are **marker classes** - they're not meant to be instantiated. They exist solely to mark parameters for injection.

Let's inspect them:

In [15]:
from langchain_core.tools import InjectedToolArg
from langgraph.prebuilt import InjectedState

print("InjectedState:")
print(f"  Type: {type(InjectedState)}")
print(f"  Is it a class? {isinstance(InjectedState, type)}")

print("\nInjectedToolCallId:")
print(f"  Type: {type(InjectedToolCallId)}")
print(f"  Is it a class? {isinstance(InjectedToolCallId, type)}")

print("\nInjectedToolArg:")
print(f"  Type: {type(InjectedToolArg)}")
print(f"  Is it a function? {callable(InjectedToolArg)}")

print("\n✅ These are marker types used in Annotated metadata!")

InjectedState:
  Type: <class 'type'>
  Is it a class? True

InjectedToolCallId:
  Type: <class 'type'>
  Is it a class? True

InjectedToolArg:
  Type: <class 'type'>
  Is it a function? True

✅ These are marker types used in Annotated metadata!


## How Annotated Works with Markers

When you write:
```python
state: Annotated[AgentState, InjectedState]
```

You're creating a type annotation where:
- **First argument** (`AgentState`): The actual type
- **Second argument** (`InjectedState`): Metadata - a marker for tooling

Let's extract this metadata manually:

In [16]:
import inspect
from typing import get_args, get_origin

# Get the signature of our tool
sig = inspect.signature(add_to_counter)

print("Inspecting add_to_counter parameters:\n")

for param_name, param in sig.parameters.items():
    annotation = param.annotation

    print(f"Parameter: {param_name}")
    print(f"  Raw annotation: {annotation}")

    # Check if it's an Annotated type
    origin = get_origin(annotation)
    if origin is not None:
        print(f"  Origin: {origin}")
        args = get_args(annotation)
        print(f"  Args: {args}")

        # Check metadata (second+ arguments)
        if len(args) > 1:
            metadata = args[1:]
            print(f"  Metadata: {metadata}")

            # Check if any metadata is an injection marker
            for meta in metadata:
                if meta is InjectedState:
                    print("    ✅ INJECTION MARKER: InjectedState")
                elif meta is InjectedToolCallId:
                    print("    ✅ INJECTION MARKER: InjectedToolCallId")
                elif isinstance(meta, type) and issubclass(
                    meta, InjectedToolArg
                ):
                    print(f"    ✅ INJECTION MARKER: {meta}")
    else:
        print(f"  Type: {annotation}")

    print()

Inspecting add_to_counter parameters:

Parameter: amount
  Raw annotation: <class 'int'>
  Type: <class 'int'>

Parameter: state
  Raw annotation: typing.Annotated[__main__.AgentState, <class 'langgraph.prebuilt.tool_node.InjectedState'>]
  Origin: <class 'typing.Annotated'>
  Args: (<class '__main__.AgentState'>, <class 'langgraph.prebuilt.tool_node.InjectedState'>)
  Metadata: (<class 'langgraph.prebuilt.tool_node.InjectedState'>,)
    ✅ INJECTION MARKER: InjectedState

Parameter: tool_call_id
  Raw annotation: typing.Annotated[str, <class 'langchain_core.tools.base.InjectedToolCallId'>]
  Origin: <class 'typing.Annotated'>
  Args: (<class 'str'>, <class 'langchain_core.tools.base.InjectedToolCallId'>)
  Metadata: (<class 'langchain_core.tools.base.InjectedToolCallId'>,)
    ✅ INJECTION MARKER: InjectedToolCallId



## ToolNode Helper Functions

LangChain and LangGraph provide helper functions to detect injected parameters. Let's explore them:

In [17]:
# Import the internal helper functions
from langchain_core.tools.base import _is_injected_arg_type
from langgraph.prebuilt.tool_node import _get_all_injected_args

print("Helper Functions Available:\n")

print("1. _is_injected_arg_type(type_) -> bool")
print("   Checks if a type annotation indicates injection")
print("   Works on: raw type annotations")
print()

print("2. _get_all_injected_args(tool) -> _InjectedArgs")
print("   Returns all injected parameters from a tool")
print("   Works on: StructuredTool objects (not raw functions!)")
print()

print("3. _is_injection(annotation) -> bool")
print("   Checks if an annotation has injection metadata")
print("   Works on: raw type annotations")
print()

print("Let's use them on our tools:")

Helper Functions Available:

1. _is_injected_arg_type(type_) -> bool
   Checks if a type annotation indicates injection
   Works on: raw type annotations

2. _get_all_injected_args(tool) -> _InjectedArgs
   Returns all injected parameters from a tool
   Works on: StructuredTool objects (not raw functions!)

3. _is_injection(annotation) -> bool
   Checks if an annotation has injection metadata
   Works on: raw type annotations

Let's use them on our tools:


In [18]:
# Test the helper functions on add_to_counter
sig = inspect.signature(add_to_counter)

print("Testing _is_injected_arg_type on each parameter:\n")

for param_name, param in sig.parameters.items():
    annotation = param.annotation
    is_injected = _is_injected_arg_type(annotation)
    print(f"{param_name:20s} -> {is_injected}")

print("\n" + "=" * 60)
print("\nUsing _get_all_injected_args:")
# Note: _get_all_injected_args expects a tool object, not a raw function
injected_args = _get_all_injected_args(tool_add_to_counter)
print(f"Return type: {type(injected_args).__name__}")
print("\nInjected parameters found:")
# _InjectedArgs is a namedtuple with fields: state, tool_call_id, store, etc.
if injected_args.state:
    print(f"  - state: {injected_args.state}")

Testing _is_injected_arg_type on each parameter:

amount               -> False
state                -> True
tool_call_id         -> True


Using _get_all_injected_args:
Return type: _InjectedArgs

Injected parameters found:
  - state: {'state': None}


## How ToolNode Executes Tools

When ToolNode runs a tool, it follows these steps:

1. **Receive tool call** from LLM (only contains non-injected args like `amount`)
2. **Detect injected parameters** using `_get_all_injected_args()`
3. **Gather injection values** from:
   - `config["configurable"]` for state
   - Tool call metadata for `tool_call_id`
4. **Merge arguments**: LLM args + injected args
5. **Execute tool** with complete arguments

Let's simulate this process:

In [19]:
# Simulate what ToolNode does

# 1. LLM provides only the non-injected args
llm_provided_args = {"amount": 10}

# 2. Detect what needs to be injected (using manual inspection)
print("ToolNode detected these need injection:")
sig = inspect.signature(add_to_counter)
injected_params = {}
for param_name, param in sig.parameters.items():
    if _is_injected_arg_type(param.annotation):
        injected_params[param_name] = param.annotation
        print(f"  {param_name}")

# 3. Gather injection values (simulated)
mock_state = {"counter": 5, "messages": []}
mock_tool_call_id = "call_abc123"

injection_values = {}
for param_name in injected_params:
    # In real ToolNode, this checks the annotation metadata
    annotation = injected_params[param_name]
    args = get_args(annotation)
    if len(args) > 1:
        for meta in args[1:]:
            if meta is InjectedState:
                injection_values[param_name] = mock_state
                print(f"  Injecting state into '{param_name}'")
            elif meta is InjectedToolCallId:
                injection_values[param_name] = mock_tool_call_id
                print(f"  Injecting tool_call_id into '{param_name}'")

# 4. Merge arguments
final_args = {**llm_provided_args, **injection_values}

print("\nFinal arguments to pass to tool:")
for key, value in final_args.items():
    if key == "state":
        print(
            f"  {key}: {type(value).__name__} (counter={value.get('counter')})"
        )
    else:
        print(f"  {key}: {value}")

# 5. Execute tool
print("\nExecuting tool...")
result = add_to_counter(**final_args)
print(f"Result: {result}")
print(f"Result: {result}")

ToolNode detected these need injection:
  state
  tool_call_id
  Injecting state into 'state'
  Injecting tool_call_id into 'tool_call_id'

Final arguments to pass to tool:
  amount: 10
  state: dict (counter=5)
  tool_call_id: call_abc123

Executing tool...
Result: Added 10 to counter. New value: 15
Result: Added 10 to counter. New value: 15


## Complete Flow Diagram

Here's the complete journey of a tool with injected arguments:

```
┌─────────────────────────────────────────────────────────────────┐
│ 1. FUNCTION DEFINITION                                          │
│                                                                 │
│   def add_to_counter(                                          │
│       amount: int,                                             │
│       state: Annotated[State, InjectedState],                 │
│       tool_call_id: Annotated[str, InjectedToolCallId]       │
│   ) -> str: ...                                                │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│ 2. CONVERT TO TOOL (@tool decorator)                           │
│                                                                 │
│   Creates StructuredTool with:                                 │
│   - args_schema: All params (including injected)              │
│   - func: Reference to original function                      │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│ 3. BIND TO MODEL (model.bind_tools([tool]))                    │
│                                                                 │
│   convert_to_anthropic_tool() calls                           │
│   convert_to_openai_tool() which:                             │
│   - Filters out injected args                                 │
│   - Returns clean schema for LLM                              │
│                                                                 │
│   Result: LLM only sees {"amount": "int"}                     │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│ 4. LLM GENERATES TOOL CALL                                      │
│                                                                 │
│   tool_call = {                                                │
│       "name": "add_to_counter",                               │
│       "args": {"amount": 10},                                 │
│       "id": "call_abc123"                                     │
│   }                                                            │
│                                                                 │
│   (No state or tool_call_id - LLM doesn't know about them)   │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│ 5. TOOLNODE EXECUTION                                           │
│                                                                 │
│   a) Receive tool_call with args={"amount": 10}              │
│   b) Call _get_all_injected_args(tool.func)                  │
│      → Returns: {"state": InjectedState,                      │
│                  "tool_call_id": InjectedToolCallId}          │
│   c) Gather values:                                           │
│      - state from config["configurable"]                      │
│      - tool_call_id from tool_call["id"]                      │
│   d) Merge: {"amount": 10,                                    │
│              "state": <actual_state>,                         │
│              "tool_call_id": "call_abc123"}                   │
│   e) Execute: tool.func(**merged_args)                        │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│ 6. RESULT                                                       │
│                                                                 │
│   "Added 10 to counter. New value: 15"                         │
└─────────────────────────────────────────────────────────────────┘
```

**Key Insight:** There are TWO phases:
- **Schema Creation** (steps 2-3): Filter out injected args
- **Runtime Execution** (step 5): Inject the values back in

## Hands-On: Build Your Own Detector

Let's build a simple function that mimics what LangChain does to detect injected parameters:

In [20]:
def detect_injected_parameters(func):
    """
    Manually detect which parameters have injection markers.
    Returns dict of {param_name: marker_type}
    """
    sig = inspect.signature(func)
    injected = {}

    for param_name, param in sig.parameters.items():
        annotation = param.annotation

        # Check if it's Annotated
        if get_origin(annotation) is not None:
            args = get_args(annotation)

            # Check metadata (args[1:])
            if len(args) > 1:
                for meta in args[1:]:
                    # Check if it's an injection marker
                    if meta is InjectedState:
                        injected[param_name] = "InjectedState"
                    elif meta is InjectedToolCallId:
                        injected[param_name] = "InjectedToolCallId"
                    elif isinstance(meta, type):
                        try:
                            if issubclass(meta, InjectedToolArg):
                                injected[param_name] = meta.__name__
                        except TypeError:
                            pass

    return injected


# Test on all our tools
print("Testing our custom detector:\n")

for tool_func in [get_counter, add_to_counter, multiply_numbers]:
    print(f"{tool_func.__name__}:")
    detected = detect_injected_parameters(tool_func)
    if detected:
        for param, marker in detected.items():
            print(f"  ✅ {param}: {marker}")
    else:
        print("  ℹ️  No injected parameters")
    print()

Testing our custom detector:

get_counter:
  ✅ state: InjectedState
  ✅ tool_call_id: InjectedToolCallId

add_to_counter:
  ✅ state: InjectedState
  ✅ tool_call_id: InjectedToolCallId

multiply_numbers:
  ℹ️  No injected parameters



## Compare: Our Detector vs LangChain's

Let's verify our implementation matches LangChain's official helpers:

In [ ]:
print("Comparing detection results:\n")

test_items = [
    ("get_counter", get_counter, tool_get_counter),
    ("add_to_counter", add_to_counter, tool_add_to_counter),
    ("multiply_numbers", multiply_numbers, tool_multiply),
]

for name, func, tool_obj in test_items:
    print(f"Function: {name}")

    # Our detector (works on raw functions)
    our_result = detect_injected_parameters(func)
    print(f"  Our detector: {list(our_result.keys())}")

    # LangChain's detector (needs tool objects)
    langchain_result = _get_all_injected_args(tool_obj)
    # Extract all parameter names from _InjectedArgs fields
    langchain_params = []

    # _InjectedArgs has fields: state, store, runtime
    # Each field is either None or a dict like {'param_name': marker}
    for field in ["state", "store", "runtime"]:
        if hasattr(langchain_result, field):
            field_value = getattr(langchain_result, field)
            if field_value and isinstance(field_value, dict):
                langchain_params.extend(field_value.keys())

    print(f"  LangChain:    {langchain_params}")

    # Compare
    if set(our_result.keys()) == set(langchain_params):
        print("  ✅ MATCH!")
    else:
        print(
            "  ℹ️  Note: _get_all_injected_args may not detect all injection types"
        )
        print("     (_is_injected_arg_type is more comprehensive)")
    print()

Comparing detection results:

Function: get_counter
  Our detector: ['state', 'tool_call_id']
  LangChain:    ['state']
  ℹ️  Note: _get_all_injected_args may not detect all injection types
     (_is_injected_arg_type is more comprehensive)

Function: add_to_counter
  Our detector: ['state', 'tool_call_id']
  LangChain:    ['state']
  ℹ️  Note: _get_all_injected_args may not detect all injection types
     (_is_injected_arg_type is more comprehensive)

Function: multiply_numbers
  Our detector: []
  LangChain:    []
  ✅ MATCH!



## Advanced: Custom Injection with InjectedToolArg

You can create custom injected arguments using `InjectedToolArg()`:

```python
from langchain_core.tools import InjectedToolArg

def my_tool(
    query: str,
    config: Annotated[dict, InjectedToolArg()]  # Custom injection!
) -> str:
    # config will be injected from RunnableConfig
    return f"Processed {query}"
```

This is useful for:
- Custom configuration objects
- Database connections
- API clients
- Any runtime context the LLM shouldn't control

## Final Summary: The Complete Picture

### Two Phases of Injected Arguments

#### Phase 1: Schema Creation (Filtering)
- **When:** Tool is bound to model via `bind_tools()`
- **What:** `convert_to_openai_tool()` filters out injected params
- **How:** Checks for `Annotated` with marker classes in metadata
- **Result:** LLM sees only "business logic" parameters

#### Phase 2: Runtime Execution (Injection)
- **When:** ToolNode executes the tool
- **What:** Injected parameters are provided automatically
- **How:** 
  1. `_get_all_injected_args()` detects what to inject
  2. Values gathered from `config["configurable"]` and tool call metadata
  3. Merged with LLM-provided args
  4. Tool executed with complete arguments
- **Result:** Function receives all parameters it needs

### Key Components

| Component | Purpose | Location |
|-----------|---------|----------|
| `InjectedState` | Mark state parameter | `langgraph.prebuilt` |
| `InjectedToolCallId` | Mark tool call ID | `langchain_core.tools` |
| `InjectedToolArg()` | Custom injection | `langchain_core.tools` |
| `_is_injected_arg_type()` | Detection helper | `langchain_core.tools.base` |
| `_get_all_injected_args()` | Get injected params | `langgraph.prebuilt.tool_node` |
| `convert_to_openai_tool()` | Filter for LLM | `langchain_core.utils.function_calling` |
| `ToolNode` | Runtime executor | `langgraph.prebuilt` |

### The Magic ✨

The brilliance of this design is **separation of concerns**:

1. **LLM** sees only what it needs to decide (user inputs)
2. **ToolNode** handles infrastructure (state, IDs, config)
3. **Your function** gets everything via clean parameters
4. **No manual plumbing** required!

You simply annotate parameters, and the framework handles the rest.